# Notebook

In [71]:
import pandas as pd
import numpy as np
import os

## Data import

In [72]:
dataset = pd.read_csv('indonesian_chat.csv')

In [73]:
dataset.head()

,id,chat,label
0,1,main mu kek tai cok,violence
1,2,user telat ngasih tau elu edan sarap gue berga...,violence
2,3,kadang berfikir percaya tuhan jatuh berkalikal...,neutral
3,4,user user aku\n\nku tau matamu sipit diliat,racist
4,5,capek deh ketemu kaum cina kapir gini match,racist


In [74]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10702 entries, 0 to 10701
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10702 non-null  int64 
 1   chat    10702 non-null  object
 2   label   10702 non-null  object
dtypes: int64(1), object(2)
memory usage: 251.0+ KB


## Data Cleaning (WIP)

In [75]:
dataset.drop("id", axis=1, inplace=True)

# Label Encode Target

In [76]:
from sklearn.preprocessing import LabelEncoder

In [77]:
le = LabelEncoder()
dataset['labels'] = le.fit_transform(dataset['label'])
dataset.drop("label", axis=1, inplace=True)

regex stuff

## BERT Import

In [78]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [79]:
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("indolem/indobert-base-uncased",num_labels=len(le.classes_))  # Number of unique labels

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Preproc

In [80]:
from sklearn.model_selection import train_test_split

In [81]:
df_train, df_test = train_test_split(dataset, test_size=0.2, random_state=42)

In [82]:
df_train.size, df_test.size

(17122, 4282)

In [83]:
df_train.head()

,chat,labels
2312,15 terimakasih selamat jalan ila jiwari robbih...,1
2516,askmf sedih gaksi orang2 ngeledekin doi gitu n...,3
3467,ape beneeeer dapet jadwal shift pagi 4 besok o...,1
8573,pantes yatim gak ngewe bareng loe,0
9814,congor nya kaum sumbu pendek,2


convert to huggingface dataset

In [84]:
from datasets import Dataset

In [85]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

## Tokenizer

In [86]:
def preprocess_text(data):
    result =  tokenizer(data["chat"],truncation=True, padding="max_length", max_length=512)
    result["labels"] = data["labels"]
    return result

In [87]:
tokenized_train = train_dataset.map(preprocess_text, batched=True)

Map:   0%|          | 0/8561 [00:00<?, ? examples/s]

Map: 100%|██████████| 8561/8561 [00:02<00:00, 3929.69 examples/s]


In [88]:
tokenized_test = test_dataset.map(preprocess_text, batched=True)

Map: 100%|██████████| 2141/2141 [00:00<00:00, 4164.74 examples/s]


In [89]:
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

## Training

In [90]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

In [91]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [92]:
import evaluate

In [93]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [94]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)


C:\Users\Warge\AppData\Local\Temp\ipykernel_20832\2964071433.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [95]:
trainer.train()

Step,Training Loss
1071,0.931400
2142,0.545300
3213,0.384500
4284,0.298000


TrainOutput(global_step=4284, training_loss=0.5398218847679046, metrics={'train_runtime': 4366.6134, 'train_samples_per_second': 7.842, 'train_steps_per_second': 0.981, 'total_flos': 9010136773607424.0, 'train_loss': 0.5398218847679046, 'epoch': 4.0})

In [96]:
trainer.save_model('filter_model')

## Evaluation

In [97]:
from sklearn.metrics import classification_report

In [98]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_train['labels'].tolist()
print(classification_report(GT,preds))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      2013
           1       0.97      0.91      0.94      2182
           2       0.96      0.98      0.97      2011
           3       0.93      0.95      0.94      2355

    accuracy                           0.95      8561
   macro avg       0.95      0.95      0.95      8561
weighted avg       0.95      0.95      0.95      8561



In [99]:
preds = trainer.predict(tokenized_test)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_test['labels'].tolist()
print(classification_report(GT,preds))

              precision    recall  f1-score   support

           0       0.88      0.91      0.90       471
           1       0.81      0.73      0.77       547
           2       0.84      0.90      0.87       495
           3       0.80      0.81      0.80       628

    accuracy                           0.83      2141
   macro avg       0.83      0.84      0.83      2141
weighted avg       0.83      0.83      0.83      2141



In [100]:
sample_dataset = dataset[:10]
hugg_sample = Dataset.from_pandas(sample_dataset)
tokenized_sample = hugg_sample.map(preprocess_text, batched=True)
predictions = trainer.predict(tokenized_sample)
predictions = np.argmax(predictions[0], axis=1)
for i, pred in enumerate(predictions):
    print(f"Chat: {sample_dataset['chat'].iloc[i]}")
    print(f"Predicted Label: {le.inverse_transform([pred])[0]}")
    print()

Map: 100%|██████████| 10/10 [00:00<00:00, 571.38 examples/s]


Chat: main mu kek tai cok
Predicted Label: violence

Chat: user telat ngasih tau elu edan sarap gue bergaul elu
Predicted Label: violence

Chat: kadang berfikir percaya tuhan jatuh berkalikali kadang tuhan ninggalkan orangtuaku berencana berpisah kakakku memilih kristen
Predicted Label: neutral

Chat: user user aku\n\nku tau matamu sipit diliat
Predicted Label: racist

Chat: capek deh ketemu kaum cina kapir gini match
Predicted Label: racist

Chat: skill cacad ngide jungler cok
Predicted Label: violence

Chat: gg main lo keren bro
Predicted Label: neutral

Chat: gue aja kelar rewatch aldnoah zero kampret emang endingnya 2 karakter utama cowonya kena friendzone bray xd url
Predicted Label: neutral

Chat: admin belanja port terbaik nak makan ai kepal milo ai kepal horlicks cendol toping kaw doket mano tu
Predicted Label: violence

Chat: user enak lg klo smbil ngewe
Predicted Label: harassment

